In [9]:
import onnx


onnx_model = onnx.load("ckpt/model.onnx")
onnx.checker.check_model(onnx_model)

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import onnx
from onnx_tf.backend import prepare

model = onnx.load('ckpt/model_5x.onnx')
tf_rep = prepare(model)

ModuleNotFoundError: No module named 'onnx_tf'

In [ ]:
import torch
import torch.onnx as torch.onnx
import onnx


import torch.nn as nn

from models.model import STBVMM

# # Initialize model with checkpointing enabled
# model = STBVMM(img_size=384, patch_size=1, in_chans=3,
#                  embed_dim=48, depths=[6, 6, 6, 6], num_heads=[6, 6, 6, 6],
#                  window_size=8, mlp_ratio=2., qkv_bias=True, qk_scale=None,
#                  drop_rate=0., attn_drop_rate=0., drop_path_rate=0.1,
#                  norm_layer=nn.LayerNorm, ape=False, patch_norm=True,
#                  use_checkpoint=True, img_range=1., resi_connection='1conv',
#                  manipulator_num_resblk = 1)

model = STBVMM(img_size=384, patch_size=1, in_chans=3,
                embed_dim=192, depths=[6, 6, 6, 6, 6, 6], num_heads=[6, 6, 6, 6, 6, 6],
                window_size=8, mlp_ratio=2., qkv_bias=True, qk_scale=None,
                drop_rate=0., attn_drop_rate=0., drop_path_rate=0.1,
                norm_layer=nn.LayerNorm, ape=False, patch_norm=True,
                use_checkpoint=False, img_range=1., resi_connection='1conv',
                manipulator_num_resblk=1)

# Load pretrained weights from checkpoint
checkpoint = torch.load('ckpt/ckpt_e49.pth.tar')
# print(checkpoint.keys())

# print(checkpoint['state_dict'])

model.load_state_dict(checkpoint['state_dict'], strict= False)

# Set the model to eval mode
model.eval()

# Export model to ONNX
inputs = (torch.randn(1, 3, 384, 384), torch.randn(1, 3, 384, 384), 5)
input_names = ["a", "b", "amp"]
output_names = ["output"]
dynamic_axes = {"a": {0: "batch_size", 2: "height", 3: "width"},
                "b": {0: "batch_size", 2: "height", 3: "width"},
                "output": {0: "batch_size", 2: "height", 3: "width"}}
onnx.export(model, inputs, "model_checkpoint_5x_50ep.onnx", input_names=input_names, output_names=output_names,
            dynamic_axes=dynamic_axes, opset_version=11)


In [6]:
import onnx

onnx_model = onnx.load("ckpt/model_checkpoint_5x_50ep.onnx")
onnx.checker.check_model(onnx_model)

In [ ]:
import onnxruntime as ort
import numpy as np
import cv2
x, y = test_data[0][0], test_data[0][1]

In [7]:
!python onnxrun.py -j4 -b1 --load_ckpt ckpt/model_checkpoint_5x_50ep.onnx --save_dir demo_video/STB-VMM_Freezer_x20_mag -m 5 --video_path demo_video/STB-VMM_Freezer_x20_original/frame --num_data 6644 --mode static



Using device: cpu
demo_video/STB-VMM_Freezer_x20_mag
processing sample: 0
Traceback (most recent call last):
  File "/Users/raoulritter/STB-VMM/onnxrun.py", line 112, in <module>
    main(args)
  File "/Users/raoulritter/STB-VMM/onnxrun.py", line 53, in main
    ort_outs = ort_session.run(None, ort_inputs)
  File "/opt/anaconda3/envs/afstudeer/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py", line 200, in run
    return self._sess.run(output_names, input_feed, run_options)
RuntimeError: Input must be a list of dictionaries or a single numpy array for input 'a'.


In [1]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)

1.11.0
0.12.0
